# Extracting embedding features from face data
In this notebook, we aim to extract embedding features from images using face recogntion extractors.
As an example, we use MOBIO dataset, and extract Arcface features from the face images:

In [5]:
##### CHANGE YOUR DATABASE HERE
from bob.bio.face.config.database.mobio_male import database
annotation_type = database.annotation_type
fixed_positions = database.fixed_positions
memory_demanding = True
dask_client = None

In [6]:
from bob.bio.face.embeddings.mxnet import arcface_insightFace_lresnet100
pipeline = arcface_insightFace_lresnet100(annotation_type=annotation_type,
                                          fixed_positions=fixed_positions,
                                          memory_demanding=memory_demanding)
transformer = pipeline.transformer

In [7]:
from bob.pipelines import wrap


features_dir = "features" #Path to store extracted features
# Wrapping with CHECKPOINT and DASK
transformer = wrap(["checkpoint","dask"],
                   transformer,
                   features_dir=features_dir)

# Printing the setup of the transformer
print(transformer)

Pipeline(steps=[('ToDaskBag', ToDaskBag()),
                ('samplewrapper-1',
                 DaskWrapper(estimator=CheckpointWrapper(estimator=SampleWrapper(estimator=FaceCrop(annotator=BobIpMTCNN(),
                                                                                                    cropped_image_size=(112,
                                                                                                                        112),
                                                                                                    cropped_positions={'leye': (55,
                                                                                                                                81),
                                                                                                                       'reye': (55,
                                                                                                                                42)}),
             

As an example, we consider 10 samples from this database and extract features for these samples:

In [8]:
# get 10 samples from database
samples = database.all_samples()[:10]

In [15]:
# Setting the DASK client
# HERE MAKE ABSOLUTELLY SURE THAT YOU DO `SETSHELL grid` 
# BEFORE STARTING THE NOTEBOOK

from dask.distributed import Client
from bob.pipelines.distributed.sge import SGEMultipleQueuesCluster

cluster = SGEMultipleQueuesCluster(min_jobs=1)
dask_client = Client(cluster)

In [16]:

features = transformer.transform(samples)
if dask_client is not None:
    features = features.compute(scheduler=dask_client)


In the following cells, we convert the extracted features to `numpy.array` and check the size of features.

In [17]:
import numpy as np
from bob.pipelines import SampleBatch

np_features = np.array(SampleBatch(features))

In [18]:
np_features

array([[[ 0.5345935 , -1.0668839 , -0.62798595, ..., -0.78859204,
         -0.5147211 ,  2.1415784 ]],

       [[ 0.24587776, -1.1436105 , -0.21513344, ..., -0.4950465 ,
         -0.7586405 ,  1.9262394 ]],

       [[-0.01235329, -1.0903177 , -0.7307515 , ..., -1.5341333 ,
         -0.9396954 ,  1.8103021 ]],

       ...,

       [[ 0.46007535, -0.9715014 , -0.52703196, ..., -0.29170716,
         -0.74297565,  1.8094344 ]],

       [[ 0.6113469 , -1.1828535 , -0.19491309, ..., -0.22889124,
         -0.58382076,  2.185493  ]],

       [[ 0.71980965, -0.4669612 , -0.49327967, ...,  0.0910981 ,
         -0.65268064,  0.93472594]]], dtype=float32)

In [19]:
# KILL THE SGE WORKERS
dask_client.shutdown()

tornado.application - ERROR - Exception in callback functools.partial(<function TCPServer._handle_connection.<locals>.<lambda> at 0x7f3470e7c8c0>, <Task finished coro=<BaseTCPListener._handle_stream() done, defined at /idiap/user/tpereira/conda/envs/bob.nightlies/lib/python3.7/site-packages/distributed/comm/tcp.py:445> exception=ValueError('invalid operation on non-started TCPListener')>)
Traceback (most recent call last):
  File "/idiap/user/tpereira/conda/envs/bob.nightlies/lib/python3.7/site-packages/tornado/ioloop.py", line 743, in _run_callback
    ret = callback()
  File "/idiap/user/tpereira/conda/envs/bob.nightlies/lib/python3.7/site-packages/tornado/tcpserver.py", line 327, in <lambda>
    gen.convert_yielded(future), lambda f: f.result()
  File "/idiap/user/tpereira/conda/envs/bob.nightlies/lib/python3.7/site-packages/distributed/comm/tcp.py", line 451, in _handle_stream
    logger.debug("Incoming connection from %r to %r", address, self.contact_address)
  File "/idiap/user/t